In [1]:
from augment import *
import glob
import os
import glob
import scipy
import random
import bisect
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import signal
from scipy.signal import find_peaks
import matplotlib.pyplot as plt
from preprocess import *                         #ماژول استخراج پنجره ها
from data import *                               #ماژول محلی ورود داده ها
from augment import *                            #ماژول های داده افزایی
from GAN import *                  
from lstm_cnn import *                           #ماژول های داده افزایی با lstm_cnn
import augment
import importlib
importlib.reload(augment)
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import BatchNormalization
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import make_classification
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report,recall_score,precision_score
from tensorflow.keras import callbacks
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Convolution2D,Conv1D,Dropout,MaxPooling1D,MaxPooling2D,Flatten,Dense
from tensorflow.keras import backend as K
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model
cwd = os.getcwd() #

In [2]:
def dstnc(dt2,dt1): #فاصله و واریانس بردار فاصله ی هر کدام از ردیف های دیتاست(آگمنت)1 از (کل)2
    dist,vrnc,i,h,w=[],[],0,np.shape(dt1)[0],np.shape(dt1)[1]
    end=min(len(dt2[0]),len(dt1[0]))
    for i in range (h):
        ds1 = dt2-dt1[i,:end] #.reshape(-1,1) #distance of every training instance
        dist_array=(ds1*10)**3  #معیار فاصله با حساسیت به توان 3 (حفظ علامت و تاثیر بیشتر تفاوت زیاد در نقاط)
        vr=np.var(ds1)
        ds1=min(np.sum(np.absolute(dist_array),axis=1))
        #print('dist2 = ', dist2)
        vrnc.append(vr)
        dist.append(ds1)
    variance=np.reshape(vrnc,(-1,1))
    distnc=np.reshape(dist,(-1,1))
    return(distnc,variance)

cls_num=2
ecg=np.array([])
for i in range (1,cls_num+1):
    vars()['ecg'+str(i)],vars()['ecg_tst'+str(i)]=TwoLeadECG(i)         #فراخوانی داده های اصلی
    ecg=np.append(ecg,vars()['ecg'+str(i)])
wdth=int(len(ecg1[0]))
ecg=np.reshape(ecg,(int(len(ecg)/wdth),wdth))  #base data windows
print('\nfinal_train_shap=',np.shape(ecg))
os.chdir(cwd)

NameError: name 'NonInvasiveFetalECGThorax1' is not defined

In [ ]:
base_wn=np.array([])
windws=np.array([])

for cls in range (1,cls_num+1):                   #ساخت پنجره های داده های آموزش اصلی
    vars()['wndws'+str(cls)]=np.array(vars()['ecg'+str(cls)])
    base_wn=np.append(base_wn,vars()['wndws'+str(cls)])
base_wn=np.reshape(base_wn,(int(len(base_wn)/wdth),wdth))  #base data windows
print("shape wndws1=", np.shape(wndws1))
print("shape base_wn=", np.shape(base_wn))

furir_windws0=np.fft.fft(np.array(base_wn[:,:-1]))
print("shape furir_windws0=", np.shape(furir_windws0))
furir_windws=np.concatenate((np.array(furir_windws0),np.transpose([base_wn[:,-1]])),axis=1)   
print("shape furir_windws=", np.shape(furir_windws))

smpl_rte=len(base_wn[0])-1                 #len(ecg1[0])  # در ماژول ها نیز همین مقدار ثبت شده
print("smpl_rte=",smpl_rte)
mx_wndws=np.max(np.bincount(np.int16(base_wn[:,-1])))
print("mx_wndws=",mx_wndws)

In [ ]:
path=('{}\AugEvl'.format(cwd)) #create folder 
if not os.path.isdir(path):
    os.mkdir(path)

def addqlty(base_wn_dt,data,mthd,i):               
    cls=int(i)
    c,alpha,PL=0,1,1
    #from glob import glob
    cwd = os.getcwd() #
    adrs=cwd+'/Aug/aug'+str(i)+str(mthd)+'.csv'
    #alldata = np.array([])                    # List for storing all the data
    train_labels=np.array(base_wn_dt[:,-1])
    tr=np.array(base_wn_dt[:,:-1])
    indx_pstv=np.where(base_wn_dt[:,-1] == i)
    P=np.array(base_wn_dt[indx_pstv])
    indx_ngtv=np.where(base_wn_dt[:,-1] != i)
    N=np.array(base_wn_dt[indx_ngtv])

    #data = np.array(data)
    aug=np.array(data)
    pnts=int(np.shape(data)[1])
    
    #aug=data[data[:,-2].argsort()]        # Positive data Sorted Base Distance
    trl=np.array(train_labels)
    for j in range (len(trl)):
        if train_labels[j]==i :           # !!!!! == (Paper GDO)
            trl[j]=1
        else :
            trl[j]=0
    trlt=np.array([trl]).T
    #mn_ds_pstv=np.reshape(aug[:,-2],(-1,1)) #min dis to positive class (submited in -2 column)
    print('shape and aug[0]=',np.shape(aug[0]),aug[0])
    print('shape P and P[0]=',np.shape(P),P[0])    
    qlty1,qlty2=mn_ds_pstv,vr_P=dstnc(P[:,1:-1],aug[:,1:-1])  #min distance to negative class
    new1=np.append(aug,qlty1,axis=1)
    new2=np.append(new1,qlty2,axis=1)
    mn_ds_ngtv,vr_N=dstnc(N[:,1:-1],aug[:,1:-1])              #min distance to negative class
    qlty3=mn_ds_pstv/mn_ds_ngtv                               #Smaller is Better
    qlty4=np.reshape(valuation(aug[:,:-1],tr,trlt),(-1,1))   #line 272
    new3=np.append(new2,qlty3,axis=1)
    new4=np.append(new3,qlty4,axis=1)
    for j in range (len(trl)):
        if train_labels[j]!=i :                          # !!!!! == (!= Inverse Paper GDO)
            trl[j]=1
        else :
            trl[j]=0
    trlt=np.array([trl]).T
    qlty5=np.reshape(valuation(aug[:,:-1],tr,trlt),(-1,1))   #line 272
    new5=np.append(new4,qlty5,axis=1)
    print('class=',i,'shape(new5)=',np.shape(new5),end='\t')
    print('mx=',np.max(new5[:,1:-7]),end='\t')
    print('mn=',np.min(new5[:,1:-7]))
    pd.DataFrame(new5[:,:]).to_csv('{}/AugEvl/AugEvl{}{}.csv'.format(cwd,cls,mthd))
    print()

In [ ]:
xtrain=np.empty((0,smpl_rte+1), float) 

for i in range (1,cls_num+1):                           #تعریف آرایه ی پنجره های کلاس ها
    vars()['ecg_aug'+str(i)+'00']=np.empty((0,smpl_rte+1), float) #Scale    
    vars()['ecg_aug'+str(i)+'10']=np.empty((0,smpl_rte+1), float) #Scale
    vars()['ecg_aug'+str(i)+'20']=np.empty((0,smpl_rte+1), float) #Mag Warp
    vars()['ecg_aug'+str(i)+'30']=np.empty((0,smpl_rte+1), float) #TimWarp
    vars()['ecg_aug'+str(i)+'40']=np.empty((0,smpl_rte+1), float) #GDO
    vars()['ecg_aug'+str(i)+'50']=np.empty((0,smpl_rte+1), float) #frequence
    vars()['ecg_aug'+str(i)+'60']=np.empty((0,smpl_rte+1), float) #GAN
    vars()['ecg_aug'+str(i)+'61']=np.empty((0,smpl_rte+1), float)
    vars()['ecg_aug'+str(i)+'62']=np.empty((0,smpl_rte+1), float)
    vars()['ecg_aug'+str(i)+'70']=np.empty((0,smpl_rte+1), float) #LSTM
    vars()['ecg_aug'+str(i)+'71']=np.empty((0,smpl_rte+1), float)
    vars()['ecg_aug'+str(i)+'72']=np.empty((0,smpl_rte+1), float)

'''f_scr=np.array([54,83,88,82,70,58,54,70])      #UnAg,Scl.2,Mag.05,Tm.2,Gs1,frqnc2,GAN,LSTMexpgdo
f_scr_difrnc=f_scr-f_scr[0]
f_scr_rtio=f_scr_difrnc/(sum(f_scr_difrnc))
aug_amnt=np.array(f_scr_rtio)                 #aug_amnt[0] is UnAg so in follow start from 1 for scale,..
'''
vlum_wndw=10*mx_wndws  

for i in range (1,cls_num+1):                           #تعریف آرایه ی پنجره های کلاس ها
    vlum_sig=10*mx_wndws-len(vars()['wndws'+str(i)])
    
    vars()['ecg_aug'+str(i)+'10']=np.append(vars()['ecg_aug'+str(i)+'10'],sclRaw(ecg,i,int(vlum_sig),mag=.2),axis=0)#0.1
    addqlty(base_wn,vars()['ecg_aug'+str(i)+'10'],10,i)
    print(str(i)+'10 =',len(vars()['ecg_aug'+str(i)+'10']))
    
    vars()['ecg_aug'+str(i)+'20']=np.append(vars()['ecg_aug'+str(i)+'20'],MgWrpRaw(ecg,i,int(vlum_sig),mag=.05),axis=0)
    addqlty(base_wn,vars()['ecg_aug'+str(i)+'20'],20,i)
    print(str(i)+'20 =',len(vars()['ecg_aug'+str(i)+'20']))
    
    vars()['ecg_aug'+str(i)+'30']=np.append(vars()['ecg_aug'+str(i)+'30'],TimWrpRaw(ecg,i,int(vlum_sig),mag=.2),axis=0)#.1
    addqlty(base_wn,vars()['ecg_aug'+str(i)+'30'],30,i)
    print(str(i)+'30 =',len(vars()['ecg_aug'+str(i)+'30']))
    
    vars()['ecg_aug'+str(i)+'40']=np.append(vars()['ecg_aug'+str(i)+'40'],GDORaw(ecg,i,int(vlum_sig),mag=.2),axis=0)
    addqlty(base_wn,vars()['ecg_aug'+str(i)+'40'],40,i)
    print(str(i)+'40 =',len(vars()['ecg_aug'+str(i)+'40']))
    
    vars()['ecg_aug'+str(i)+'50']=np.append(vars()['ecg_aug'+str(i)+'50'],frqncRaw(furir_windws,i,int(vlum_sig),mag=.2),axis=0)
    addqlty(base_wn,vars()['ecg_aug'+str(i)+'50'],50,i)
    print(str(i)+'50 =',len(vars()['ecg_aug'+str(i)+'50']))

In [ ]:
for i in range (1,cls_num+1):                           #تعریف آرایه ی پنجره های کلاس ها
    aug_pls60,aug_pls61,aug_pls62=3,4,5 #2 is MgWrp and 4 is GDO and 5 is freq Augmentes Data for GAN Training
    vars()['ecg_aug'+str(i)+'60']=np.append(vars()['ecg_aug'+str(i)+'60'],GAN_Gnrt(ecg,i,aug_pls60),axis=0)
    addqlty(base_wn,vars()['ecg_aug'+str(i)+'60'],60,i)
    print(str(i)+'60 =',len(vars()['ecg_aug'+str(i)+'60']))
    vars()['ecg_aug'+str(i)+'61']=np.append(vars()['ecg_aug'+str(i)+'61'],GAN_Gnrt(ecg,i,aug_pls61),axis=0)
    addqlty(base_wn,vars()['ecg_aug'+str(i)+'61'],61,i)
    print(str(i)+'61 =',len(vars()['ecg_aug'+str(i)+'61']))
    #xtrain=np.append(xtrain,vars()['ecg_aug60'+str(i)],axis=0)
    vars()['ecg_aug'+str(i)+'62']=np.append(vars()['ecg_aug'+str(i)+'62'],GAN_Gnrt(ecg,i,aug_pls62),axis=0)
    addqlty(base_wn,vars()['ecg_aug'+str(i)+'62'],62,i)
    print(str(i)+'62 =',len(vars()['ecg_aug'+str(i)+'62']))
    #xtrain=np.append(xtrain,vars()['ecg_aug60'+str(i)],axis=0)

In [ ]:
for i in range (1,cls_num+1):                           #تعریف آرایه ی پنجره های کلاس ها
    aug_pls70,aug_pls71,aug_pls72=3,4,5 #2 is MgWrp and 3 is TimeWrp Augmentes Data for LSTM Augmentation
    vars()['ecg_aug'+str(i)+'70']=np.append(vars()['ecg_aug'+str(i)+'70'],LSTM_Gnrt_Pridc(ecg,i,aug_pls70),axis=0)
    addqlty(base_wn,vars()['ecg_aug'+str(i)+'70'],70,i)
    print(str(i)+'70 =',len(vars()['ecg_aug'+str(i)+'70']))
    vars()['ecg_aug'+str(i)+'71']=np.append(vars()['ecg_aug'+str(i)+'71'],LSTM_Gnrt_Pridc(ecg,i,aug_pls71),axis=0)
    addqlty(base_wn,vars()['ecg_aug'+str(i)+'71'],71,i)
    print(str(i)+'71 =',len(vars()['ecg_aug'+str(i)+'71']))
    vars()['ecg_aug'+str(i)+'72']=np.append(vars()['ecg_aug'+str(i)+'72'],LSTM_Gnrt_Pridc(ecg,i,aug_pls72),axis=0)
    addqlty(base_wn,vars()['ecg_aug'+str(i)+'72'],72,i)
    print(str(i)+'72 =',len(vars()['ecg_aug'+str(i)+'72']))